# FunSearch Demo:

**In this notebook you will:**
- Learn how to use FunSearch on the task of solving codeforces problems [Section 1](#section-1-solving-codeforces-problems-with-funsearch)
- Learn the various ways to interact with FunSearch [Section 1](#section-1-solving-codeforces-problems-with-funsearch)
- Learn how to adapt FunSearch to your own problem solving needs [Section 2](#section-2-adapting-funsearch-to-your-own-problem)


## What is FunSearch?

FunSearch, born from a paper featured in Nature by DeepMind, can be succinctly characterized as follows, mirroring their website's description:

> 'FunSearch, a method to search for new solutions in mathematics and computer science. FunSearch works by pairing a pre-trained LLM, whose goal is to provide creative solutions in the form of computer code, with an automated “evaluator”, which guards against hallucinations and incorrect ideas. By iterating back-and-forth between these two components, initial solutions “evolve” into new knowledge. The system searches for “functions” written in computer code; hence the name FunSearch.'  - [[1]](https://deepmind.google/discover/blog/funsearch-making-new-discoveries-in-mathematical-sciences-using-large-language-models/) [[2]](https://www.nature.com/articles/s41586-023-06924-6)

**Pre-requisites:**

We strongly recommend reading the original paper [[2]](https://www.nature.com/articles/s41586-023-06924-6) for a more in-depth understanding of FunSearch.

[1] DeepMind, 2023. ["FunSearch: Making new discoveries in mathematical sciences using Large Language Models"](https://deepmind.google/discover/blog/funsearch-making-new-discoveries-in-mathematical-sciences-using-large-language-models/)

[2] [Romera-Paredes, Bernardino, Mohammadamin Barekatain, Alexander Novikov, Matej Balog, M. Pawan Kumar, Emilien Dupont, Francisco JR Ruiz et al. "Mathematical discoveries from program search with large language models." Nature 625, no. 7995 (2024): 468-475](https://www.nature.com/articles/s41586-023-06924-6)

## Imports, Dependencies, and Starting a Local Server (No action required, just run the cells)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import hydra
import aiflows
from aiflows.backends.api_info import ApiInfo
from aiflows.utils.general_helpers import read_yaml_file, quick_load_api_keys
from aiflows.utils import serving
from aiflows.workers import run_dispatch_worker_thread
from aiflows.utils.colink_utils import start_colink_server
from aiflows import flow_verse
from aiflows.utils import logging
#setting verbosity to warning because of the large amount of logs (seems to create issues in notebook)
logging.set_verbosity_warning()
import json
from aiflows.utils import serving
import os

#LoadFunSearch
dependencies = [
    {
        "url": "aiflows/FunSearchFlowModule",
        "revision": "main"
    }
]
flow_verse.sync_dependencies(dependencies)

!pip install -r flow_modules/aiflows/FunSearchFlowModule/pip_requirements.txt

from utils import make_best_program_per_island_nice, dict_to_yaml, compile_and_writefile, download_codeforces_data, get_configs_cf
import pandas as pd


/Users/nicolasbaldwin/opt/miniconda3/envs/mockenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fetching 36 files: 100%|██████████| 36/36 [00:00<00:00, 144079.15it/s]


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 64860.37it/s]


In [3]:
cl = start_colink_server()

## Section 1: Solving Codeforces Problems with FunSearch

### Downloading Codeforces Dataset

Let's start by downloading the codeforces dataset and taking a look at the first few rows. The full details of the dataset can be found [here](https://github.com/epfl-dlab/cc_flows?tab=readme-ov-file#codeforces).

In [13]:
if not os.path.exists("./data/codeforces.jsonl.gz"):
    download_codeforces_data("./data", "codeforces.jsonl.gz")

In [14]:
#Taking a Look at DS
df = pd.read_json("./data/codeforces.jsonl.gz", lines=True, compression='gzip')

In [15]:
df.head(3)

,contest,problem_name,problem_fullname,problem_url,id,difficulty,tags,header,problem_description,input_description,...,hidden_tests_io_truncated,num_tests_non_truncated,solutions,solution_url,working_solution,id_hash,time_limit,memory_limit,interpolated_date,release_date
0,1600,J,J. Robot Factory,https://codeforces.com/contest/1600/problem/J,1600J,1400,"[bitmasks, dfs and similar]",J. Robot Factorytime limit per test1 secondmem...,You have received data from a Bubble bot. You ...,Input\nThe first line has two numbers which ar...,...,"[[[917 992, 13 15 9 14 15 15 11 12 11 14 15 9 ...",2,"[{'verdict': 'Accepted', 'language': 'PyPy 3-6...",https://codeforces.com/contest/1600/submission...,import sys\ninput = sys.stdin.readline\n \n \n...,0c63a6eae6aefee1587c318ed6fbab140ab440b07575e4...,1 second,256 megabytes,None,2021-10-09
1,1638,B,B. Odd Swap Sort,https://codeforces.com/contest/1638/problem/B,1638B,1100,"[data structures, math, sortings]",B. Odd Swap Sorttime limit per test1 secondmem...,"You are given an array a_1, a_2, \dots, a_n. Y...",Input\nEach test contains multiple test cases....,...,"[[[33954, 1, 2, 1, 1, 2, 2 4, 2, 2 2, 2, 1 2, ...",1,"[{'verdict': 'Accepted', 'language': 'PyPy 3-6...",https://codeforces.com/contest/1638/submission...,"for s in[*open(0)][2::2]:\n a=[[],[]]\n for x ...",16faeda04f84a9f7f54640aa51249fcdc9cf264617373c...,1 second,256 megabytes,None,2022-02-14
2,1638,A,A. Reverse,https://codeforces.com/contest/1638/problem/A,1638A,800,"[constructive algorithms, greedy, math]",A. Reversetime limit per test1 secondmemory li...,"You are given a permutation p_1, p_2, \ldots, ...",Input\nEach test contains multiple test cases....,...,"[[[500, 1, 1, 2, 1 2, 2, 2 1, 3, 1 2 3, 3, 1 3...",1,"[{'verdict': 'Accepted', 'language': 'Python 3...",https://codeforces.com/contest/1638/submission...,"for s in[*open(0)][2::2]:\n a=[*map(int,s.spli...",f031d386d7a22ba8571134c260aebd7b20f6cb9795b897...,1 second,256 megabytes,None,2022-02-14


Each Row in the dataset is a codeforces problem with a unique id in the `id` column.

In [16]:
problem_id = "1789B"
prob = df[df.id == problem_id]
display(df[df.id == problem_id])
print(f"Problem Statement:\n{prob.problem_description.values[0]}")

,contest,problem_name,problem_fullname,problem_url,id,difficulty,tags,header,problem_description,input_description,...,hidden_tests_io_truncated,num_tests_non_truncated,solutions,solution_url,working_solution,id_hash,time_limit,memory_limit,interpolated_date,release_date
142,1789,B,B. Serval and Inversion Magic,https://codeforces.com/contest/1789/problem/B,1789B,800,"[brute force, implementation, strings, two poi...",B. Serval and Inversion Magictime limit per te...,Serval has a string s that only consists of 0 ...,Input\nEach test contains multiple test cases....,...,"[[[10000, 2, 00, 2, 01, 2, 10, 2, 11, 3, 000, ...",1,"[{'verdict': 'Accepted', 'language': 'PyPy 3-6...",https://codeforces.com/contest/1789/submission...,class solve:\n def __init__(self):\n ...,c5c18430aba9033582c658845be4146bcbf13ccf522fa2...,1 second,256 megabytes,None,2023-02-25


Problem Statement:
Serval has a string s that only consists of 0 and 1 of length n. The i-th character of s is denoted as s_i, where 1\leq i\leq n.
Serval can perform the following operation called Inversion Magic on the string s:
Choose an segment [l, r] (1\leq l\leq r\leq n). For l\leq i\leq r, change s_i into 1 if s_i is 0, and change s_i into 0 if s_i is 1.
For example, let s be 010100 and the segment [2,5] is chosen. The string s will be 001010 after performing the Inversion Magic.
Serval wants to make s a palindrome after performing Inversion Magic exactly once. Help him to determine whether it is possible.
A string is a palindrome iff it reads the same backwards as forwards. For example, 010010 is a palindrome but 10111 is not.


### Solving a Codeforces Problem with FunSearch


Let's employ FunSearch to tackle a Codeforces problem. 

#### Serving, and Getting an Instance of FunSearch

To facilitate this process, we've developed a helper function named `get_configs_cf`, which is responsible for setting up the configuration files and providing an initial "dummy solution" for FunSearch to address a designated Codeforces problem -— in this case, problem ID 1789B. We'll delve into the details of how to configure these configuration files in Section 2.

In [8]:
problem_id = "1789B"
funsearch_cfg, dummy_solution = get_configs_cf(problem_id,ds_location="./data/codeforces.jsonl.gz")

In [9]:
print(dummy_solution)

def solve_function(input) -> str:
    """Attempt at solving the problem given the input input and returns the predicted output (see the top of the file for problem description)"""
    return 'YES'



In [10]:
print(json.dumps(funsearch_cfg, indent=4))

{
    "name": "FunSearchFlow",
    "description": "A flow implementing FunSearch Asynchronous Search",
    "subflows_config": {
        "SamplerFlow": {
            "name": "Sampler Flow",
            "description": "A flow that queries an LLM model to generate prompts",
            "system_message_prompt_template": {
                "partial_variables": {
                    "evaluate_name": "evaluate",
                    "evolve_name": "solve_function",
                    "artifacts_per_prompt": 2
                }
            },
            "backend": {
                "api_infos": "???"
            }
        },
        "EvaluatorFlow": {
            "name": "A flow that evaluates code on tests",
            "description": "A flow that evaluates code on tests",
            "singleton": false,
            "run_error_score": -1,
            "py_file": "\"\"\"Problem Description:\nServal has a string s that only consists of 0 and 1 of length n. The i-th character of s is denoted as s

Now let's start serving FunSearch:

In [11]:
serving.recursive_serve_flow(
        cl=cl,
        flow_class_name="flow_modules.aiflows.FunSearchFlowModule.FunSearch",
        flow_endpoint="FunSearch",
)

True

We can also configure how many workers we want to use for FunSearch:

In [12]:
n_workers = 10
for i in range(n_workers):
    run_dispatch_worker_thread(cl)

Finally, we can get an instance of FunSearch set up to solve problem 1789B:

In [13]:

api_information = [ApiInfo(backend_used="openai",api_key = os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(funsearch_cfg, api_information, key="api_infos")
funsearch_proxy = serving.get_flow_instance(
        cl=cl,
        flow_endpoint="FunSearch",
        config_overrides=funsearch_cfg,
)

#### Interacting with FunSearch

In this subsection we will be showing how to interact with FunSearch. More specifically, we will be showing how to:
- Register a solution to FunSearch (i.e., provide a solution to FunSearch, typically a dummy solution)
- Get the best solutions produced by FunSearch
- Start the search process
- Stop the search process


##### Registering a Solution

To get started with FunSearch, the first thing you need to do is register a solution. This can be a simple placeholder solution that FunSearch will try to make better. Just remember, FunSearch won't be able to begin searching without a solution provided.

In [14]:
##How to Register a Program to FunSearch

#Register the first program
#You can register a program by sending a message to the FunSearch flow (pretending be a SamplerFlow)
data = {
        "from": "SamplerFlow", #Keep this as is
        "operation": "register_program", #Keep this as is
        "api_output": dummy_solution #This is the solution you want to register
    }
#Package the message
input_message = funsearch_proxy.package_input_message(data = data)
#send the message (without expecting a response)
funsearch_proxy.send_message(input_message)


##### Get the best solutions produced by FunSearch

Here below we show how to get the best solutions produced by FunSearch. Since we haven't actually started FunSearch and have only registered a dummy solution, we will only see the dummy solution.

The `get_best_solutions` method returns a list of dictionaries containing the best solutions found by FunSearch so far on each island of the ProgramDB. Each item in the list is a dictionary with the following keys:
- `rank`: The rank of the solution (among the best solutions found by FunSearch). Best rank is 1
- `score`: The score of the solution (for codeforces problems, the maximal score is 1.0)
- `island_id`: The id of the island where the solution was found in the ProgramDB
- `program`: The solution itself

In [15]:
#Fetch Best Programs
data = {
        "from": "FunSearch", #Keep this as is
        "operation": "get_best_programs_per_island", #Keep this as is
        "content": {} #Keep this as is
}
#Package the message
input_message = funsearch_proxy.package_input_message(data = data)
#send the message (expecting a response)
future = funsearch_proxy.get_reply_future(input_message)
print("waiting for response....")
#wait for the response (this will block until the response is received)
response = future.get_data()
#Print the response
nice_response = make_best_program_per_island_nice(response) #This is a helper function to make the response more readable
print("\n\n".join(nice_response))

waiting for response....

### Rank: 3
### Score: 0.25
### Island ID: 0
### Program:
def solve_function(input) -> str:
    """Attempt at solving the problem given the input input and returns the predicted output (see the top of the file for problem description)"""
    return 'YES'





### Rank: 2
### Score: 0.25
### Island ID: 1
### Program:
def solve_function(input) -> str:
    """Attempt at solving the problem given the input input and returns the predicted output (see the top of the file for problem description)"""
    return 'YES'





### Rank: 1
### Score: 0.25
### Island ID: 2
### Program:
def solve_function(input) -> str:
    """Attempt at solving the problem given the input input and returns the predicted output (see the top of the file for problem description)"""
    return 'YES'





##### Start the search process of FunSearch

Here below is how to start the search process of FunSearch. FunSearch will try to improve the solution you provided.

In [16]:
#Start FunSearch
data = {
        "from": "FunSearch", #Keep this as is
        "operation": "start", #Keep this as is
        "content": {"num_samplers": 5}, #This is the number of samplers you want to start
    }
#NOTE: You are not assured that all the samplers will start. 
#       The number of samplers that will start is dependent on the number of available workers

#Package the message
input_message = funsearch_proxy.package_input_message(data = data)
#send the message (without expecting a response)
funsearch_proxy.send_message(input_message)


You can keep on checking the best solutions produced by FunSearch by calling `get_best_solutions` method (just run the cell below).
In my experience, **FunSearch takes not more than a couple minutes to find an optimal solution** for this codeforces problem.

In [22]:
#Fetch Best Programs
data = {
        "from": "FunSearch",
        "operation": "get_best_programs_per_island",
        "content": {}
}
    
input_message = funsearch_proxy.package_input_message(data = data)

future = funsearch_proxy.get_reply_future(input_message)
print("waiting for response....")
response = future.get_data()
nice_response = make_best_program_per_island_nice(response)
print("\n".join(nice_response))

waiting for response....

### Rank: 3
### Score: 0.5
### Island ID: 0
### Program:
def solve_function_v1(input) -> str:
    """Improved version of solve_function_v0"""
    t = int(input[0])
    results = []
    for i in range(t):
        n = int(input[i*2+1])
        s = input[i*2+2]
        if s == s[::-1]:
            results.append('YES')
        else:
            results.append('NO')
    return ' '.join(results)




### Rank: 2
### Score: 0.5
### Island ID: 1
### Program:
def solve_function_v2(input) -> str:
    """Improved version of solve_function_v1"""
    def is_palindrome(s):
        return s == s[::-1]

    def solve_case(s):
        zero_count = s.count('0')
        one_count = s.count('1')
        if zero_count % 2 == 1 and one_count % 2 == 1:
            return 'NO'
        else:
            return 'YES' if is_palindrome(s) else 'NO'

    t = int(input[0])
    results = []
    for i in range(t):
        n = int(input[i*2+1])
        s = input[i*2+2]
        results.append(

##### Stop the search process of FunSearch

Here below is how to stop the search process of FunSearch. More specifally, it will let samplers who are ongoing to finish their work and then stop the search process. Note that you can always start the search process again by calling `start` method (see above).

In [23]:
#Stop FunSearch
input_message = funsearch_proxy.package_input_message(data = data)
    
funsearch_proxy.send_message(input_message)

data = {
    "from": "FunSearch",
    "operation": "stop",
    "content": {},
}

input_message = funsearch_proxy.package_input_message(data = data)

funsearch_proxy.send_message(input_message)

## Section 2: Adapting FunSearch to your own Problem

In this section, we will show how to adapt FunSearch to your own problem. We will showcase this by adapting FunSearch to solve a problem presented in the paper by DeepMind [[2]](https://www.nature.com/articles/s41586-023-06924-6): The capset problem.

For details on the capset problem, please refer to the original paper [[2]](https://www.nature.com/articles/s41586-023-06924-6) or [Wikipedia](https://en.wikipedia.org/wiki/Cap_set)


### Defining your own problem

The first thing you need to do is to **define your evaluate in a py file**. You can chose the name of your evaluate function. In the example below we chose the name `evalute`. This function should take as input:
- `function_str`: The function to evaluate passed as a string
- any other arguments you need. In this case, we add the `n` too
and return a score. Note that you can define multiple functions that the evaluate function can call.

Then you must **define a dummy solution of your evolve function in a seperate py file**. You can chose the name of your evolve function. In the example below we chose the name `priority`. The inputs of this function depend on your problem. In this case, the input is `el` corresponding to a candidate vector to be added to the capset.

In [17]:
%%compile_and_writefile ./Capset_problem/evaluate_functions.py

"""Finds large cap sets."""

import ast
import itertools
import numpy as np
from typing import List, Callable

### !!! Don't Touch this Function !!! ###
def get_function_name_from_code(code):
    tree = ast.parse(code)
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            return True, node.name

    # something is wrong
    return False, None

### !!! Don't Touch this Function !!! ###
def get_function(function_str: str):
    local_namespace = {}
    
    exec(function_str,local_namespace)
    found_name, program_name = get_function_name_from_code(function_str)
    
    if not found_name:
        raise ValueError(f"Function name not found in program: {function_str}")
    
    priority_fn = local_namespace.get(program_name)
    return priority_fn

# Define Your evaluate function here
def evaluate(function_str: str, n: int) -> int:
    """Returns the size of an `n`-dimensional cap set."""
    
    # This line must always be in evaluate function (converts generated string sample to callable)
    priority_fn = get_function(function_str) 
    capset = solve(priority_fn,n)
    return len(capset)

# Solve function, used in evaluate
def solve(priority_fn: Callable, n: int) -> np.ndarray:
    """Returns a large cap set in `n` dimensions."""
    
    all_vectors = np.array(list(itertools.product((0, 1, 2), repeat=n)), dtype=np.int32)

    # Powers in decreasing order for compatibility with `itertools.product`, so
    # that the relationship `i = all_vectors[i] @ powers` holds for all `i`.
    powers = 3 ** np.arange(n - 1, -1, -1)

    # Precompute all priorities.
    priorities = np.array([priority_fn(tuple(vector), n) for vector in all_vectors])

    # Build `capset` greedily, using priorities for prioritization.
    capset = np.empty(shape=(0, n), dtype=np.int32)
    while np.any(priorities != -np.inf):
        # Add a vector with maximum priority to `capset`, and set priorities of
        # invalidated vectors to `-inf`, so that they never get selected.
        max_index = np.argmax(priorities)
        vector = all_vectors[None, max_index]  # [1, n]
        blocking = np.einsum('cn,n->c', (- capset - vector) % 3, powers)  # [C]
        priorities[blocking] = -np.inf
        priorities[max_index] = -np.inf
        capset = np.concatenate([capset, vector], axis=0)

    return capset






In [18]:
%%compile_and_writefile ./Capset_problem/evolve_functions.py

# The function we wish to evolve with Funsearch
def priority(el: tuple[int, ...], n: int) -> float:
  """Returns the priority with which we want to add `element` to the cap set."""
  return 0.0

### Loading your own problem to FunSearch

Next we will show how to change the configuration files in order to run FunSearch on your own problem.

You should start by loading the functions you just defined in your py files. You can use the `Loader` class from the `FunSearchFlowModule` to do so:

In [19]:
from flow_modules.aiflows.FunSearchFlowModule.Loader import Loader

# path where we saved the evaluate function
path_to_evaluate_functions_file = os.path.join(".", "Capset_problem", "evaluate_functions.py")
# name we gave to the evaluate function
name_of_evaluate_function = "evaluate"
# path where we saved the evolve function
path_to_evolve_functions_file = os.path.join(".", "Capset_problem", "evolve_functions.py")
# name we gave to the evolve function
name_of_evolve_function = "priority"

loader = Loader(file_path = path_to_evaluate_functions_file, target_name = name_of_evaluate_function)
evaluate_function = loader.load_target()
evaluate_file_full_content = loader.load_full_file()

loader = Loader(file_path = path_to_evolve_functions_file, target_name = name_of_evolve_function)
evolve_function = loader.load_target()




Now that we've loaded the functions, we can define our configuration file for the problem and adjust any parameters we need.

You can override more stuff: Check out https://huggingface.co/aiflows/FunSearchFlowModule for full configuration parameters

In [20]:
#Note:
artifacts_per_prompt = 2
overrides = {
    "subflows_config": {
        #Override configuration of SamplerFlow
        "SamplerFlow": {
            #Arguments that must be passed to the system prompt template
            "system_message_prompt_template": {
                "partial_variables": {
                    "evaluate_name": name_of_evaluate_function,
                    "evolve_name": name_of_evolve_function,
                    "artifacts_per_prompt": artifacts_per_prompt
                }
            },
            "backend": {
                "api_infos": "???" # We will add API info after
            }
        },
        "EvaluatorFlow": {
            #Score to give to a program that fails to run
            "run_error_score": -1,
            #Necessary code to run evaluation function 
            "py_file": evaluate_file_full_content,
            #Name of the evaluate function
            "function_to_run_name": name_of_evaluate_function,
            #Inputs to evaluate function on which we will score (can be more than one test)
            "test_inputs": {
                "n=3": {
                    "n": 3,
                },
            },
            #Timeout in seconds for each evaluation
            "timeout_seconds": 10,
        },
        "ProgramDBFlow": {
            #evaluate function to use to evaluate programs
            "evaluate_function": evaluate_function,
            #Full content of evaluate function
            "evaluate_file_full_content": evaluate_file_full_content,
            #Name of function to evolve
            "artifact_to_evolve_name": name_of_evolve_function,
            #Number of islands. !! Default for FunSearch experiments was 10 but we are using 3 for this example
            "num_islands": 3,
            #Number of evolve examples from previous generations to include in prompt
            "artifacts_per_prompt": artifacts_per_prompt
        }
    }
}
print(json.dumps(overrides, indent=4))

{
    "subflows_config": {
        "SamplerFlow": {
            "system_message_prompt_template": {
                "partial_variables": {
                    "evaluate_name": "evaluate",
                    "evolve_name": "priority",
                    "artifacts_per_prompt": 2
                }
            },
            "backend": {
                "api_infos": "???"
            }
        },
        "EvaluatorFlow": {
            "run_error_score": -1,
            "py_file": "\n\"\"\"Finds large cap sets.\"\"\"\n\nimport ast\nimport itertools\nimport numpy as np\nfrom typing import List, Callable\n\n###\u00a0!!! Don't Touch this Function !!! ###\ndef get_function_name_from_code(code):\n    tree = ast.parse(code)\n    for node in ast.walk(tree):\n        if isinstance(node, ast.FunctionDef):\n            return True, node.name\n\n    # something is wrong\n    return False, None\n\n### !!! Don't Touch this Function !!! ###\ndef get_function(function_str: str):\n    local_namespace = 

In [21]:
#### !!!!!!! Uncomment code below if you didn't run section 1 !!!!!! ####
# cl = start_colink_server()
# n_workers = 10
# for i in range(n_workers):
#     run_dispatch_worker_thread(cl)
    
# serving.recursive_serve_flow(
#         cl=cl,
#         flow_class_name="flow_modules.aiflows.FunSearchFlowModule.FunSearch",
#         flow_endpoint="FunSearch",
# )

#Load api information
api_information = [ApiInfo(backend_used="openai",api_key = os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(overrides, api_information, key="api_infos")
custom_funsearch_proxy = serving.get_flow_instance(
        cl=cl,
        flow_endpoint="FunSearch",
        config_overrides=overrides,
)

#### Register dummy solution

In [22]:
##How to Register a Program to FunSearch

#Register the first program
#You can register a program by sending a message to the FunSearch flow (pretending be a SamplerFlow)
data = {
        "from": "SamplerFlow", #Keep this as is
        "operation": "register_program", #Keep this as is
        "api_output": evolve_function #This is the solution you want to register
    }
#Package the message
input_message = custom_funsearch_proxy.package_input_message(data = data)
#send the message (without expecting a response)
custom_funsearch_proxy.send_message(input_message)

#### Start the search process

In [23]:
#Start FunSearch
data = {
        "from": "FunSearch", #Keep this as is
        "operation": "start", #Keep this as is
        "content": {"num_samplers": 5}, #This is the number of samplers you want to start
    }
#NOTE: You are not assured that all the samplers will start. 
#       The number of samplers that will start is dependent on the number of available workers

#Package the message
input_message = custom_funsearch_proxy.package_input_message(data = data)
#send the message (without expecting a response)
custom_funsearch_proxy.send_message(input_message)

#### Query Best Solutions (Run as many times as you like)

For cost reasons, we will not run the search process here. But you can run the search process by calling the `start` method.

In [30]:
#Fetch Best Programs
data = {
        "from": "FunSearch", #Keep this as is
        "operation": "get_best_programs_per_island", #Keep this as is
        "content": {} #Keep this as is
}
#Package the message
input_message = custom_funsearch_proxy.package_input_message(data = data)
#send the message (expecting a response)
future = custom_funsearch_proxy.get_reply_future(input_message)
print("waiting for response....")
#wait for the response (this will block until the response is received)
response = future.get_data()
#Print the response
nice_response = make_best_program_per_island_nice(response) #This is a helper function to make the response more readable
print("\n\n".join(nice_response))

waiting for response....



### Rank: 3
### Score: 8.0
### Island ID: 0
### Program:
def priority(el: tuple[int, ...], n: int) -> float:
    """Returns the priority with which we want to add `element` to the cap set."""
    return 0.0





### Rank: 2
### Score: 8.0
### Island ID: 1
### Program:
def priority(el: tuple[int, ...], n: int) -> float:
    """Returns the priority with which we want to add `element` to the cap set."""
    return 0.0





### Rank: 1
### Score: 8.0
### Island ID: 2
### Program:
def priority(el: tuple[int, ...], n: int) -> float:
    """Returns the priority with which we want to add `element` to the cap set."""
    return 0.0





#### Stop the search process

In [29]:
#Stop FunSearch
input_message = custom_funsearch_proxy.package_input_message(data = data)
    
custom_funsearch_proxy.send_message(input_message)

data = {
    "from": "FunSearch",
    "operation": "stop",
    "content": {},
}

input_message = custom_funsearch_proxy.package_input_message(data = data)

custom_funsearch_proxy.send_message(input_message)